# API GitHub

In [66]:
import requests 

In [67]:
r = requests.get('https://api.github.com/events')

In [68]:
r 

<Response [200]>

In [69]:
r = requests.get('https://api.github.com/versions')
r.status_code

200

In [70]:
r.json()

['2022-11-28']

# Extract

In [71]:
# especificando vrssão da API 
headers = {'X-GitHub-Api-Versions':'2022-11-28'}

In [72]:
api_base_url = 'https://api.github.com'
owner = 'amzn' # user de quem vamos extrair os dados
url = f'{api_base_url}/users/{owner}/repos'


In [73]:
url

'https://api.github.com/users/amzn/repos'

In [74]:
response = requests.get(url, headers=headers)
response.status_code

200

In [75]:
len(response.json())

30

# Authorization

In [ ]:
import os
access_token = os.getenv('TOKEN_GITHUB')
headers = {'Authorization': f'Bearer {access_token}','X-GitHub-Api-Versions':'2022-11-28'}

# Pagination

In [77]:
#lista de paginas + .json()
repos_list = [] # lista composta por sub listas.
for page_num in range(1,6):
    params = {'page': page_num,'per_page': 100} # de 100 em 100 ate a 5 page.
    try:
        response = requests.get(url,headers=headers,params=params)
        if response.status_code == 200:
            repos_list.append(response.json())
        else:
            print(f'Página {page_num} retornou status {response.status_code}')
            repos_list.append(None)
    except Exception as e:
        print(f'Erro ao acessar página {page_num}: {e}')
        repos_list.append(None)

In [78]:
len(repos_list) # paginas

5

In [79]:
len(repos_list[0]) # conteudo

100

# Transform 

In [80]:
repos_list[0][4]['name'] # abre uma das listas e checa dados(name)

'alexa-coho'

In [81]:
repos_name = []
for page in repos_list: # lista com varias listas
    if page is not None:
        for repo in page: # dentro de uma das listas
            repos_name.append(repo['name']) # adiciona o nome dentro de cada uma das listas da lista de listas.



In [82]:
len(repos_name) # checando se pegou todos os repositorios da API.

162

In [83]:
repos_name[:10]

['.github',
 'ads-advanced-tools-docs',
 'ads-pao-amznjs-gtm-server-side-template',
 'ads-pao-amznjs-gtm-template',
 'alexa-coho',
 'alexa-skills-kit-js',
 'amazon-ads-advertiser-audience-normalization-sdk-py',
 'amazon-advertising-api-php-sdk',
 'amazon-codeguru-profiler-for-spark',
 'amazon-frustration-free-setup-certification-tool']

In [84]:
repos_lang = []
for page in repos_list:
    if page is not None:
        for repos in page:
            repos_lang.append(repos['language'])

In [85]:
repos_lang[:5]

[None, 'Jupyter Notebook', 'Smarty', 'Smarty', 'JavaScript']

In [86]:
len(repos_lang)

162

# DataFrame

In [87]:
import pandas as pd

In [88]:
dados_amz = pd.DataFrame() # fazendo o DF
dados_amz['repository Name'] = repos_name # criando coluna para lista de names.
dados_amz['Languages'] = repos_lang # coluna de linguagens usadas.

In [89]:
# checagem 
dados_amz

,repository Name,Languages
0,.github,None
1,ads-advanced-tools-docs,Jupyter Notebook
2,ads-pao-amznjs-gtm-server-side-template,Smarty
3,ads-pao-amznjs-gtm-template,Smarty
4,alexa-coho,JavaScript
...,...,...
157,zeek-plugin-enip,Zeek
158,zeek-plugin-profinet,Zeek
159,zeek-plugin-s7comm,Zeek
160,zeek-plugin-tds,Zeek


In [90]:
dados_amz.to_csv('amazon.csv')

# Load
## via API no repositorios do github

In [91]:
api_base_url = 'https://api.github.com' # api de antes
url = f'{api_base_url}/user/repos'
url

'https://api.github.com/user/repos'

In [92]:
# criando repositorio POST
data = {
    'name':'linguagens_utilizadas',
    'description':'Repositorio com linguages de Prog Amazon',
    'private': False # true == privado | false == publico
}

response = requests.post(url, json=data, headers=headers) # esse codigo vai criar o repositorio com as caractericas de data
response.status_code

201

In [93]:
# colocando o arquivo no formato base64
import base64 

In [94]:
with open('amazon.csv', 'rb') as file: # lê o arquivo csv em binario
    file_content = file.read()

encoded_content = base64.b64encode(file_content) #arquivo base64 da DF na variavel enconded_content

In [95]:
#subindo o arquivo para o repositorio
api_base_url = 'https://api.github.com'
username = 'FireFox-exe'
repo = 'linguagens_utilizadas'
path = 'amazon.csv'

url = f'{api_base_url}/repos/{username}/{repo}/contents/{path}'
url

'https://api.github.com/repos/FireFox-exe/linguagens_utilizadas/contents/amazon.csv'

In [96]:
# PUT
data = {
    'message': 'adicionando novo arquivo',
    'content': encoded_content.decode('utf-8') # de base4 para texto
}

response = requests.put(url, json=data, headers=headers)
response.status_code

201